# Semantic Kernel

In this code sample, you will use the [**Semantic Kernel**](https://aka.ms/ai-agents-beginners/semantic-kernel) **AI** framework to create a basic agent.
The goal of this sample is to show you the steps that we will later use in the additional code samples when implementing the different agentic patterns.

## Prepare the project

Import necessary packages:

In [1]:
import os
import json

Import necessary entities:

In [ ]:
from random import choice
from typing import Annotated
from dotenv import load_dotenv
from openai import AsyncOpenAI
from json import JSONDecodeError
from IPython.display import HTML, display
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.contents import (
    FunctionCallContent,
    StreamingTextContent,
    FunctionResultContent,
)

Load enviroment for using a system variables:

In [3]:
load_dotenv();

## Prepare agent functionality

In this sample, we will use [**GitHub** models](https://aka.ms/ai-agents-beginners/github-models) for access to the **LLM**.
The `ai_model_id` is defined as `gpt-4o-mini`.
Try changing the model to another model available on the **GitHub** models marketplace to see the different results.
For us to use the `Azure Inference SDK` that is used for the `base_url` for **GitHub** models, we will use the `OpenAIChatCompletion` connector within **Semantic Kernel**.
There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use **Semantic Kernel** for other model providers.

Prepare a list of vacation destinations:

In [4]:
vacation_destinations: list[str] = [
    "Tokyo, Japan",
    "Cairo, Egypt",
    "Paris, France",
    "New York, USA",
    "Berlin, Germany",
    "Bali, Indonesia",
    "Barcelona, Spain",
    "Sydney, Australia",
    "Rio de Janeiro, Brazil",
    "Cape Town, South Africa",
]

Create a plugin class:

In [5]:
class DestinationsPlugin:
    """
    A plugin that manages and provides vacation destinations.

    :Attributes:
        last_destination (str | None): The last destination that was returned.
                                       Default: None.
        vacation_destinations (list[str]): The list of available destinations.
    """

    def __init__(self, vacation_destinations: list[str]) -> None:
        """
        Initializes the `DestinationsPlugin` with a list of destinations.

        :Parameters:
            vacation_destinations (list[str]): The list of available
                                               destinations.
        """

        self.last_destination: str | None = None
        self.vacation_destinations: list[str] = vacation_destinations

    @kernel_function(description="Returns a random vacation destination.")
    def get_random_vacation_destination(
        self
    ) -> Annotated[str, "Returns a random vacation destination."]:
        """
        Selects and returns a random vacation destination from the available
        options.

        :Returns:
            str: A randomly selected vacation destination.
        """

        available_destinations: list[str] = self.vacation_destinations.copy()

        if self.last_destination and (len(available_destinations, ) > 1):
            available_destinations.remove(self.last_destination, )

        self.last_destination = choice(available_destinations, )

        return self.last_destination

## Create a client

Create a client instance:

In [6]:
client: AsyncOpenAI = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN", ),
    base_url="https://models.inference.ai.azure.com/",
)

Create an **AI** service that will be used by the `ChatCompletionAgent`:

In [7]:
chat_completion_service: OpenAIChatCompletion = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creating the agent

Below we create the agent called `TravelAgent`.
For this example, we are using very simple instructions.
You can change these instructions to see how the agent responds differently.

In [8]:
agent: ChatCompletionAgent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[
        DestinationsPlugin(vacation_destinations=vacation_destinations, ),
    ],
    name="TravelAgent",
    instructions="You are a helpful AI agent that can help plan vacations for" +
                 " customers at random destinations.",
)

## Running the agent

Now we can run the agent by defining the `ChatHistory` and adding the `system_message` to it.
We will use the `AGENT_INSTRUCTIONS` that we defined earlier.
After these are defined, we create a `user_inputs` that will be what the user is sending to the agent.
In this case, we have set this message to `Plan me a sunny vacation.`.
Feel free to change this message to see how the agent responds differently.

Create an user inputs:

In [9]:
user_inputs: list[str] = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

Create a workflow function:

In [ ]:
async def main() -> None:
    """
    Executes a streaming conversation with an AI agent, rendering both text
    responses and function calls in an interactive HTML format.

    :Raises:
        JSONDecodeError: If function arguments are malformed JSON
        RuntimeError: If streaming fails or HTML rendering unavailable
    """

    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        argument_buffer: str = ""
        html_output: str = (
            f"<div style='margin-bottom:10px'>" +
            f"<div style='font-weight:bold'>User:</div>" +
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )
        agent_name: str | None = None
        full_response: list[str] = []
        function_calls: list[str] = []
        current_function_name: str | None = None

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items: list = list(response.items, )

            for item in content_items:
                if isinstance(item, FunctionCallContent, ):
                    if item.function_name:
                        current_function_name = item.function_name

                    if isinstance(item.arguments, str, ):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent, ):
                    if current_function_name:
                        formatted_args: str = argument_buffer.strip()

                        try:
                            parsed_args: dict = json.loads(formatted_args, )
                            formatted_args = json.dumps(parsed_args, )
                        except JSONDecodeError as json_dec_err:
                            function_calls.append(f"⚠️ Invalid JSON arguments: {str(e)}")
                            formatted_args = argument_buffer  # Fallback: show raw text
                        except TypeError as e:
                            function_calls.append(f"⚠️ Arguments must be a string: {str(e)}")

                        function_calls.append(
                            "Calling function:" +
                            f"{current_function_name}({formatted_args})",
                        )

                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(
                        f"\nFunction Result:\n\n{item.result}",
                    )
                elif isinstance(item, StreamingTextContent, ) and item.text:
                    full_response.append(item.text, )

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output, ), )


Launch the agent workflow:

In [11]:
await main()